In [1]:
import os
import re
import pandas as pd
from joblib import load
import ru_core_news_sm as ru

In [2]:
%%time
tfidf = load('tfidf.joblib')    
lr = load('logreg.joblib')
top_vectorizer = load('top100.joblib')

lem = ru.load()

CPU times: total: 32.2 s
Wall time: 33.3 s


In [3]:
def convert(text):
    text = text.lower()
    text = " ".join(re.sub(r'[^u"а-яА-Я"]', ' ', text).split()) #удаляем не буквы
    text = " ".join(re.sub(r'(^|\s)\w{1}(\s\w{1})*($|\s)', ' ', text).split()) #удаляем слова из 1 буквы
    
    doc = lem(text)
    text = " ".join(token.lemma_ for token in doc)
    return text

In [4]:
def predict(text):
    text = convert(text)
    features = tfidf.transform(pd.Series(text))
    top_words = top_vectorizer.transform(pd.Series(text))
    return lr.predict_proba(features)[0][0], top_words.count_nonzero()
# Первый параметр вероятность пренадлежности к 0 классу
# Второй параметр количество слов из топ 100 слов решений (<50 не является решением)